In [1]:
!pip install openpyxl

In [7]:
import pandas as pd
import numpy as np

import torch as t
import torch.nn.functional as f

from tqdm.notebook import trange, tqdm

from bs4 import BeautifulSoup

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

# Импорт и обработка данных

In [4]:
data = pd.read_excel('/content/sample_data/dataset_comments.xlsx').drop(['UserSenderId', 'SubmitDate'], axis=1)

In [5]:
from bs4 import BeautifulSoup

# Удалим html теги
def html_to_text(html):
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = text.replace("\xa0", " ")
    return text.strip()

data.MessageText = data.MessageText.apply(lambda x: html_to_text(x))

In [6]:
import re

# Частично избавимся от "шума"
def is_noise(text):
    if len(text) < 3:  # Убираем слишком короткие строки
        return True
    if re.match(r"^[\W\d_]+$", text):  # Только символы и цифры
        return True
    if re.match(r"^[a-zA-Z]+$", text):  # Только латиница
        return True
    return False

data = data[~data.MessageText.apply(is_noise)]